In [20]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

artists_data_path = "../data/artists.jsonl"
sessions_data_path = "../data/sessions.jsonl"
track_storage_data_path = "../data/track_storage.jsonl"
tracks_data_path = "../data/tracks.jsonl"
users_data_path = "../data/users.jsonl"

artists_data_frame = pd.read_json(artists_data_path, lines=True)
sessions_data_frame = pd.read_json(sessions_data_path, lines=True)
tracks_data_frame = pd.read_json(tracks_data_path, lines=True)
users_data_frame = pd.read_json(users_data_path, lines=True)

# Scalenie danych

In [21]:
merged_data_frame = pd.merge(sessions_data_frame, tracks_data_frame, left_on="track_id",
                             right_on="id")
merged_data_frame = pd.merge(merged_data_frame, users_data_frame, left_on="user_id",
                             right_on="user_id")
merged_data_frame = pd.merge(merged_data_frame, artists_data_frame, left_on="id_artist",
                             right_on="id")
# rename column
merged_data_frame = merged_data_frame.rename(columns={"name_x": "track_name"})
merged_data_frame = merged_data_frame.rename(columns={"name_y": "artist_name"})
merged_data_frame.head(5)


,timestamp,user_id,track_id,event_type,session_id,id_x,track_name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,mode,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_name,city,street,favourite_genres,premium_user,id_y,name,genres
0,2023-01-04 00:12:59.000,101,0NPjiwqT1xrA3ck05xKoA8,PLAY,124,0NPjiwqT1xrA3ck05xKoA8,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,1975,0.667,0.199,1,NaN,-19.284,0.952,0.470,0.0,0.0882,0.487,96.391,3,Klara Herdzik,Szczecin,pl. Floriana 55/22,"[permanent wave, mandopop, funk]",False,1c6OwPjqCGGUg770n3zhbq,Margaretha Krook,[barnsagor]
1,2023-02-04 01:20:47.302,926,0NPjiwqT1xrA3ck05xKoA8,PLAY,11284,0NPjiwqT1xrA3ck05xKoA8,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,1975,0.667,0.199,1,NaN,-19.284,0.952,0.470,0.0,0.0882,0.487,96.391,3,Dagmara Łaszczyk,Szczecin,pl. Lawendowa 35/83,"[alternative metal, rock, c-pop]",False,1c6OwPjqCGGUg770n3zhbq,Margaretha Krook,[barnsagor]
2,2023-02-04 01:22:54.580,926,0NPjiwqT1xrA3ck05xKoA8,LIKE,11284,0NPjiwqT1xrA3ck05xKoA8,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,1975,0.667,0.199,1,NaN,-19.284,0.952,0.470,0.0,0.0882,0.487,96.391,3,Dagmara Łaszczyk,Szczecin,pl. Lawendowa 35/83,"[alternative metal, rock, c-pop]",False,1c6OwPjqCGGUg770n3zhbq,Margaretha Krook,[barnsagor]
3,2023-01-28 02:29:05.362,926,1hviQqMhM4NyY4O6CWZABO,PLAY,11281,1hviQqMhM4NyY4O6CWZABO,"Det finns väl ingen med kniv här i stan, del 3",20,113858,0,1c6OwPjqCGGUg770n3zhbq,1992-10-30,0.676,0.330,8,1.0,-16.983,0.954,0.737,0.0,0.1890,0.301,79.707,4,Dagmara Łaszczyk,Szczecin,pl. Lawendowa 35/83,"[alternative metal, rock, c-pop]",False,1c6OwPjqCGGUg770n3zhbq,Margaretha Krook,[barnsagor]
4,2023-01-28 02:30:48.635,926,1hviQqMhM4NyY4O6CWZABO,SKIP,11281,1hviQqMhM4NyY4O6CWZABO,"Det finns väl ingen med kniv här i stan, del 3",20,113858,0,1c6OwPjqCGGUg770n3zhbq,1992-10-30,0.676,0.330,8,1.0,-16.983,0.954,0.737,0.0,0.1890,0.301,79.707,4,Dagmara Łaszczyk,Szczecin,pl. Lawendowa 35/83,"[alternative metal, rock, c-pop]",False,1c6OwPjqCGGUg770n3zhbq,Margaretha Krook,[barnsagor]


## Wyliczenie etykiety, czy dana piosenka w danej sesji została pominięta przez danego użytkownika

In [22]:
# delete rows with event_type other than PLAY and SKIP
merged_data_frame = merged_data_frame[merged_data_frame["event_type"].isin(["PLAY", "SKIP"])]

# if for one user_id and track_id and session_id there is only PLAY event, then the track was not skipped
merged_data_frame["skipped"] = merged_data_frame.groupby(["user_id", "track_id", "session_id"])[
  "event_type"].transform(lambda x: x != "PLAY").astype(bool)

# if there are 2 rows with the same user_id, track_id and session_id, and different "skipped" value, remove the row with "skipped" = 0
merged_data_frame = merged_data_frame.drop_duplicates(subset=["user_id", "track_id", "session_id"],
                                                      keep="last")



## Dodanie nowych atrybutów

In [23]:
merged_data_frame["number_of_matching_genres"] = merged_data_frame.apply(lambda x: len(set(x["genres"]).intersection(set(x["favourite_genres"]))), axis=1)
merged_data_frame["month"] = merged_data_frame["timestamp"].dt.month
merged_data_frame["day_of_week"] = merged_data_frame["timestamp"].dt.dayofweek
merged_data_frame["hour_of_day"] = merged_data_frame["timestamp"].dt.hour
merged_data_frame["genres_with_favourite_genres"] = merged_data_frame["genres"] + merged_data_frame["favourite_genres"]
merged_data_frame["common_genres"] = merged_data_frame["genres_with_favourite_genres"].apply(lambda x: list(set(x)))

## Usunięcie niepotrzebnych kolumn

In [24]:
merged_data_frame = merged_data_frame.drop(columns=["id_x", "id_y", "event_type", "mode"])
merged_data_frame.head(10)

,timestamp,user_id,track_id,session_id,track_name,popularity,duration_ms,explicit,id_artist,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist_name,city,street,favourite_genres,premium_user,name,genres,skipped,number_of_matching_genres,month,day_of_week,hour_of_day,genres_with_favourite_genres,common_genres
0,2023-01-04 00:12:59.000,101,0NPjiwqT1xrA3ck05xKoA8,124,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,1975,0.667,0.199,1,-19.284,0.952,0.470,0.0,0.0882,0.487,96.391,3,Klara Herdzik,Szczecin,pl. Floriana 55/22,"[permanent wave, mandopop, funk]",False,Margaretha Krook,[barnsagor],False,0,1,2,0,"[barnsagor, permanent wave, mandopop, funk]","[permanent wave, mandopop, barnsagor, funk]"
1,2023-02-04 01:20:47.302,926,0NPjiwqT1xrA3ck05xKoA8,11284,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,1975,0.667,0.199,1,-19.284,0.952,0.470,0.0,0.0882,0.487,96.391,3,Dagmara Łaszczyk,Szczecin,pl. Lawendowa 35/83,"[alternative metal, rock, c-pop]",False,Margaretha Krook,[barnsagor],False,0,2,5,1,"[barnsagor, alternative metal, rock, c-pop]","[rock, barnsagor, alternative metal, c-pop]"
4,2023-01-28 02:30:48.635,926,1hviQqMhM4NyY4O6CWZABO,11281,"Det finns väl ingen med kniv här i stan, del 3",20,113858,0,1c6OwPjqCGGUg770n3zhbq,1992-10-30,0.676,0.330,8,-16.983,0.954,0.737,0.0,0.1890,0.301,79.707,4,Dagmara Łaszczyk,Szczecin,pl. Lawendowa 35/83,"[alternative metal, rock, c-pop]",False,Margaretha Krook,[barnsagor],True,0,1,5,2,"[barnsagor, alternative metal, rock, c-pop]","[rock, barnsagor, alternative metal, c-pop]"
6,2023-02-14 21:59:20.619,940,0NPjiwqT1xrA3ck05xKoA8,11473,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,1975,0.667,0.199,1,-19.284,0.952,0.470,0.0,0.0882,0.487,96.391,3,Stanisław Butrym,Wrocław,ulica Witosa 13,"[ranchera, pop, latin alternative]",True,Margaretha Krook,[barnsagor],True,0,2,1,21,"[barnsagor, ranchera, pop, latin alternative]","[latin alternative, barnsagor, pop, ranchera]"
7,2023-02-06 18:41:13.873,1095,0NPjiwqT1xrA3ck05xKoA8,13459,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,1975,0.667,0.199,1,-19.284,0.952,0.470,0.0,0.0882,0.487,96.391,3,Fabian Zbroja,Wrocław,pl. Orzechowa 97/37,"[singer-songwriter, pop rock, post-teen pop]",True,Margaretha Krook,[barnsagor],False,0,2,0,18,"[barnsagor, singer-songwriter, pop rock, post-teen pop]","[post-teen pop, barnsagor, pop rock, singer-songwriter]"
8,2023-01-12 06:42:49.000,1295,0NPjiwqT1xrA3ck05xKoA8,16327,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,1975,0.667,0.199,1,-19.284,0.952,0.470,0.0,0.0882,0.487,96.391,3,Klara Majzner,Gdynia,plac Witosa 89/15,"[pop, pop, funk]",True,Margaretha Krook,[barnsagor],False,0,1,3,6,"[barnsagor, pop, pop, funk]","[funk, barnsagor, pop]"
11,2023-02-15 23:36:35.331,1467,0NPjiwqT1xrA3ck05xKoA8,18574,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,1975,0.667,0.199,1,-19.284,0.952,0.470,0.0,0.0882,0.487,96.391,3,Aurelia Miąsko,Poznań,plac Wierzbowa 28,"[post-teen pop, argentine rock, latin rock]",True,Margaretha Krook,[barnsagor],True,0,2,2,23,"[barnsagor, post-teen pop, argentine rock, latin rock]","[argentine rock, latin rock, barnsagor, post-teen pop]"
14,2023-03-16 19:24:41.953,1544,0NPjiwqT1xrA3ck05xKoA8,19590,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,1975,0.667,0.199,1,-19.284,0.952,0.470,0.0,0.0882,0.487,96.391,3,Dariusz Tokarek,Kraków,pl. Toruńska 266,"[europop, filmi, alternative metal]",True,Margaretha Krook,[barnsagor],True,0,3,3,19,"[barnsagor, europop, filmi, alternative metal]","[europop, barnsagor, alternative metal, filmi]"
16,2023-03-14 16:20:47.409,2039,0NPjiwqT1xrA3ck05xKoA8,26304,"Anton skaffar sig hund, del 4",21,159812,0,1c6OwPjqCGGUg770n3zhbq,1975,0.667,0.199,1,-19.284,0.952,0.470,0.0,0.0882,0.487,96.391,3,Bruno Jeszke,Szczecin,al. Kamienna 457,"[mpb, soul, alternative rock]",True,Margaretha Kroo

In [25]:
merged_data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1276510 entries, 0 to 2179841
Data columns (total 35 columns):
 #   Column                        Non-Null Count    Dtype         
---  ------                        --------------    -----         
 0   timestamp                     1276510 non-null  datetime64[ns]
 1   user_id                       1276510 non-null  int64         
 2   track_id                      1276510 non-null  object        
 3   session_id                    1276510 non-null  int64         
 4   track_name                    1276510 non-null  object        
 5   popularity                    1276510 non-null  int64         
 6   duration_ms                   1276510 non-null  int64         
 7   explicit                      1276510 non-null  int64         
 8   id_artist                     1276510 non-null  object        
 9   release_date                  1276510 non-null  object        
 10  danceability                  1276510 non-null  float64       
 11  ene

In [26]:
merged_data_frame.describe()

,timestamp,user_id,session_id,popularity,duration_ms,explicit,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,number_of_matching_genres,month,day_of_week,hour_of_day
count,1276510,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06,1.276510e+06
mean,2023-02-17 06:30:55.759123968,2.859073e+03,3.724297e+04,4.637075e+01,2.309341e+05,8.232603e-02,5.665693e-01,5.997763e-01,5.279286e+00,-9.098161e+00,8.181135e-02,3.240204e-01,7.121924e-02,2.046845e-01,5.397278e-01,1.204185e+02,3.930228e+00,1.659979e-01,2.095890e+00,2.974657e+00,1.150624e+01
min,2023-01-01 00:04:29,1.010000e+02,1.240000e+02,0.000000e+00,4.000000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-6.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,2023-01-24 11:10:13.823000064,1.486000e+03,1.883100e+04,2.600000e+01,1.802830e+05,0.000000e+00,4.550000e-01,4.310000e-01,2.000000e+00,-1.148100e+01,3.350000e-02,3.610000e-02,0.000000e+00,9.420000e-02,3.410000e-01,9.854400e+01,4.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,6.000000e+00
50%,2023-02-17 05:30:08.064999936,2.869000e+03,3.729600e+04,4.100000e+01,2.149470e+05,0.000000e+00,5.730000e-01,6.200000e-01,5.000000e+00,-8.315000e+00,4.250000e-02,2.120000e-01,4.410000e-05,1.260000e-01,5.470000e-01,1.199650e+02,4.000000e+00,0.000000e+00,2.000000e+00,3.000000e+00,1.200000e+01
75%,2023-03-13 01:19:03.110000128,4.234000e+03,5.576400e+04,7.500000e+01,2.622400e+05,0.000000e+00,6.860000e-01,7.890000e-01,9.000000e+00,-6.050000e+00,7.680000e-02,5.740000e-01,5.200000e-03,2.470000e-01,7.430000e-01,1.368920e+02,4.000000e+00,0.000000e+00,3.000000e+00,5.000000e+00,1.700000e+01
max,2023-04-05 20:24:55.388000,5.600000e+03,7.413100e+04,9.600000e+01,4.120258e+06,1.000000e+00,9.820000e-01,1.000000e+00,1.100000e+01,2.487000e+00,9.670000e-01,9.960000e-01,9.990000e-01,1.000000e+00,1.000000e+00,2.180720e+02,5.000000e+00,3.000000e+00,4.000000e+00,6.000000e+00,2.300000e+01
std,NaN,1.589681e+03,2.135982e+04,2.537431e+01,9.897527e+04,2.748609e-01,1.657664e-01,2.278273e-01,3.559949e+00,4.344660e+00,1.169348e-01,3.107948e-01,2.033808e-01,1.895666e-01,2.515016e-01,2.848244e+01,3.764612e-01,4.159961e-01,9.210477e-01,2.009431e+00,6.903861e+00


## Współczynnik informacji wzajemnej

Sprawdzamy, czy atrybuty niosą ze sobą informację o zmiennej celu.

In [27]:
from sklearn import metrics
from scipy.stats import entropy

print("Entropy of skipped column: ", str(round(entropy(merged_data_frame["skipped"].value_counts()), 4)))

df = merged_data_frame.copy()
df["favourite_genres"] = df["favourite_genres"].apply(lambda x: str(x))
df["genres"] = df["genres"].apply(lambda x: str(x))
df["genres_with_favourite_genres"] = df["genres_with_favourite_genres"].apply(lambda x: str(x))
df["common_genres"] = df["common_genres"].apply(lambda x: str(x))

mutual_info_score = {}

for column in df.columns:
  if column != "skipped":
    mutual_info_score[column] = round(metrics.mutual_info_score(df["skipped"], df[column]), 4)

# print sorted by value
for key, value in sorted(mutual_info_score.items(), key=lambda item: item[1], reverse=True):
  print("%s: %s" % (key, value))

Entropy of skipped column:  0.6523


/Users/rasztabigab/anaconda3/envs/IUM/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/Users/rasztabigab/anaconda3/envs/IUM/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/Users/rasztabigab/anaconda3/envs/IUM/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/Users/rasztabigab/anaconda3/envs/IUM/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received binary v

timestamp: 0.4952
genres_with_favourite_genres: 0.4764
common_genres: 0.4551
session_id: 0.171
user_id: 0.1391
street: 0.139
artist_name: 0.1386
favourite_genres: 0.1357
premium_user: 0.0344
track_id: 0.0151
track_name: 0.0147
tempo: 0.0136
duration_ms: 0.0119
loudness: 0.0095
id_artist: 0.0082
name: 0.0082
release_date: 0.0064
genres: 0.0063
danceability: 0.005
instrumentalness: 0.0047
acousticness: 0.0042
valence: 0.0033
speechiness: 0.0027
energy: 0.0024
liveness: 0.0024
popularity: 0.0004
city: 0.0002
explicit: 0.0001
key: 0.0001
month: 0.0001
time_signature: 0.0
number_of_matching_genres: 0.0
day_of_week: 0.0
hour_of_day: 0.0


## Macierz korelacji

In [28]:
df = df.drop(columns=["genres", "favourite_genres", "genres_with_favourite_genres", "common_genres", "track_id", "track_name", "id_artist", "release_date", "artist_name", "city", "street", "name"])

df["skipped"] = df["skipped"].astype(int)
df["premium_user"] = df["premium_user"].astype(int)
df["timestamp"] = pd.to_numeric(df["timestamp"], errors="coerce")

correlation_matrix = df.corrwith(df["skipped"])
correlation_matrix

timestamp                   -0.013453
user_id                      0.003888
session_id                   0.003906
popularity                  -0.006764
duration_ms                  0.052508
explicit                    -0.010313
danceability                -0.075586
energy                      -0.000453
key                          0.003896
loudness                    -0.003847
speechiness                 -0.030642
acousticness                -0.007098
instrumentalness             0.013742
liveness                     0.009499
valence                     -0.045396
tempo                        0.011392
time_signature              -0.003515
premium_user                 0.262185
skipped                      1.000000
number_of_matching_genres    0.003511
month                       -0.012886
day_of_week                  0.001368
hour_of_day                 -0.002158
dtype: float64

In [29]:
# save to jsonl file
merged_data_frame.to_json("../data/merged_data.jsonl", orient="records", lines=True)


## Podsumowanie

Jak widać na podstawie współczyników informacji wzajemnej, połączenie atrybutów "genres" i "favourite_genres" jest dobrym atrybutem do uczenia naszego modelu predykcyjnego.
Innymi dobrymi atrybutami są: "tempo", "duration_ms" i "loudness".

Początkowo zdecydowaliśmy się na użycie atrybutów "genres" i "favourite_genres" jako danych wejściowych modelu. Zmienną celu jest etykieta "skipped".
